In [199]:
#^ Essential Imports
import logging
import configparser
import pandas as pd
import pandas_ta as ta
import robin_stocks as rstocks
from robin_stocks import robinhood as r
from datetime import datetime
from pytz import timezone
import alive_progress
import traceback
from alive_progress import alive_bar
from icecream import ic
import asyncio
import sys
import time
import os
import random
from tqdm import tqdm
PCT_SPEND = 0.05 # 5% of buying power
verbose_mode = True # Set to True to see more logging output
from ratelimit import limits, sleep_and_retry
from colorama import Fore, Back, Style

In [200]:
#^ load the relevant variables from the .ini credentials file in config/
#^ Starting out with the basics
config = configparser.ConfigParser()
config.read('config/credentials.ini')
coins = config['trading']['coins'].split(', ')
stop_loss_percent = float(config['trading']['stop_loss_percent'])
coins = [coin.strip() for coin in coins]
percent_to_use = float(config['trading']['percent_to_use'])
verbose_mode = config['logging']['verbose_mode']
debug_verbose = config['logging']['debug_verbose']
reset_positions = config['logging']['reset_positions']
minimum_usd_per_position = float(config['trading']['minimum_usd_per_position']) #note: this is the minimum amount of USD that must be invested at any given time in each position. All trades must keep this in mind.
# get pct_to_buy_with from config file
pct_to_buy_with = config['trading']['percent_to_use'] #*fixed
pct_to_buy_with = float(pct_to_buy_with)
pct_to_buy_per_trade = config['trading']['percent_to_spend_per_trade'] #* fixed
pct_to_buy_per_trade = float(pct_to_buy_per_trade)

In [201]:
#^ Logging Setup
if not os.path.exists('logs'):
    os.makedirs('logs')
logging.basicConfig(filename='logs/robinhood.log', level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

In [202]:
"""
Notes
r.crypto.get_crypto_quote(position['currency']['code'])['mark_price']
use the format above to get the current price of a coin in USD

ordering crypto should use this syntax:
    r.orders.order_crypto(
        symbol = coin,
        amountIn = 'dollars', # or 'quantity'
        side = 'buy',
        quantityOrPrice = buy_cost,#buy_cost / float(df.close.iloc[-1]), # this is the amount of the coin to buy in units of the coin
        limitPrice = df.close.iloc[-1],
        timeInForce = 'gtc',
        jsonify = True
    )

Increments apply to each kind of coin
    r.crypto.get_crypto_info('BTC')['min_order_size'] # this is the minimum amount of BTC that can be bought at a time
# Get crypto positions
positions = r.crypto.get_crypto_positions()
print(f'found {len(positions)} positions.')
for position in tqdm(positions):
    # print(position)
    if position['currency']['code'] == crypto:
        pos_dict = position['currency']
        min_order_size = float(pos_dict['increment'])
        coin_holdings = float(position['quantity_available'])
"""

"\nNotes\nr.crypto.get_crypto_quote(position['currency']['code'])['mark_price']\nuse the format above to get the current price of a coin in USD\n\nordering crypto should use this syntax:\n    r.orders.order_crypto(\n        symbol = coin,\n        amountIn = 'dollars', # or 'quantity'\n        side = 'buy',\n        quantityOrPrice = buy_cost,#buy_cost / float(df.close.iloc[-1]), # this is the amount of the coin to buy in units of the coin\n        limitPrice = df.close.iloc[-1],\n        timeInForce = 'gtc',\n        jsonify = True\n    )\n\nIncrements apply to each kind of coin\n    r.crypto.get_crypto_info('BTC')['min_order_size'] # this is the minimum amount of BTC that can be bought at a time\n# Get crypto positions\npositions = r.crypto.get_crypto_positions()\nprint(f'found {len(positions)} positions.')\nfor position in tqdm(positions):\n    # print(position)\n    if position['currency']['code'] == crypto:\n        pos_dict = position['currency']\n        min_order_size = float(p

In [203]:

#! Format for a Buy Order
# buy_order_response = r.orders.order_crypto(
#     symbol = coin,
#     amountIn = 'dollars', # or 'quantity'
#     side = 'buy',
#     quantityOrPrice = buy_cost,#buy_cost / float(df.close.iloc[-1]), # this is the amount of the coin to buy in units of the coin
#     limitPrice = rounded_limit_price,
#     timeInForce = 'gtc',
#     jsonify = True
# )
#! Format for a Sell Order
# # Submit an order using the r.orders.order_sell_crypto_limit() function
# sell_order_response = r.orders.order_crypto(
#     symbol = coin,
#     amountIn = 'quantity', # in coins
#     side = 'sell',
#     quantityOrPrice = volume_to_sell_usd / float(df.close.iloc[-1]), # this is the amount of the coin to sell in units of the coin
#     limitPrice = df.close.iloc[-1],
#     timeInForce = 'gtc',
#     jsonify = True
# )
# r.crypto.get_crypto_quote(position['currency']['code'])['mark_price']
# use the format above to get the current price of a coin in USD


In [204]:
# Purpose: Redo the Robinhood API trader in Python
#* Step One - Log in to the Robinhood API
username = config['robinhood']['username']
password = config['robinhood']['password']
login = r.login(username, password)
print(f"Logged in as {username}")

Logged in as "graham.waters37@gmail.com"


In [205]:
print(f'Checking Available Buying Power...')
total_money = float(r.load_account_profile()['crypto_buying_power']) * pct_to_buy_with
print(f'Available Buying Power: ${total_money}', end='')
print(f' * {pct_to_buy_with} = ${total_money * pct_to_buy_with}')
available_money = total_money * pct_to_buy_with

Checking Available Buying Power...


Available Buying Power: $75.736 * 0.8 = $60.588800000000006


In [294]:
# Action Function -- Buy a Coin
# Input: coin name, Amount in USD, Current Price of Coin
@sleep_and_retry
def get_current_price(coin_name):
    # Get a current quote for the coin in question
    quote = r.crypto.get_crypto_quote(coin_name)
    # Get the current price of the coin
    current_price = quote['mark_price']
    return current_price

@sleep_and_retry
def update_portfolio():
    # Assuming you've already logged in
    positions = r.crypto.get_crypto_positions()
    positions_dict = {}
    for position in positions:
        # print(f"You own {position['quantity']} of {position['currency']['code']}")
        positions_dict[position['currency']['code']] = float(position['quantity'])
    return positions_dict

@sleep_and_retry
def buy_coin(
    coin_name = 'BTC',
    amount = 1.00,
    current_price = 10000
    ):
    # Get the current price of the coin
    current_price = get_current_price(coin_name)
    # Calculate the amount of coins to buy
    amount_to_buy = float(amount) / float(current_price)
    # Buy the coin and return the amount bought (round the buy price to 2 decimal places)
    buying_usd = round(float(amount_to_buy), 2)
    buy_cost = round(float(buying_usd) * float(current_price), 2)
    result = r.orders.order_crypto(
        symbol = coin_name,
        amountIn = 'dollars', # or 'quantity'
        side = 'buy',
        quantityOrPrice = float(buy_cost),#buy_cost / float(df.close.iloc[-1]), # this is the amount of the coin to buy in units of the coin
        limitPrice = float(current_price),
        timeInForce = 'gtc',
        jsonify = True
    )
    return result

@sleep_and_retry
def sell_coin(
    coin_name = 'BTC',
    ):
    positions_dict = update_portfolio()
    # Get a current quote for the coin in question
    quote = r.crypto.get_crypto_quote(coin_name)
    quote_value = quote['mark_price']
    quote_float = float(quote_value)
    quote = round(quote_float, 2)
    # Get the current price of the coin
    current_price = float(get_current_price(coin_name))
    # Calculate the amount of coins to sell
    # Sell All Coins of this type in the portfolio
    amount_held = float(positions_dict[coin_name]) 
    # Sell the coin and return the amount sold (round the sell price to 2 decimal places)
    # selling_usd = round(amount_held * current_price, 2)
    # sell_cost = round(float(selling_usd) / float(current_price), 2) 
    result = r.orders.order_crypto(
        symbol = coin_name,
        amountIn = 'quantity', # or 'quantity'
        side = 'sell', # this is a sell order
        quantityOrPrice = float(amount_held), # this is the amount of the coin to buy in units of the coin
        limitPrice = float(current_price),
        timeInForce = 'gtc', # good till cancelled
        jsonify = True # return json
    )
    return result


basic_orders = {}
positions_dict = update_portfolio()
# amount_held = float(positions_dict[coin]) 
# fill in the basic orders dictionary with 1.00 USD buys of each coin in the coins list
for coin in tqdm(coins):
    basic_orders[coin] = [buy_coin(coin, 1.00, float(get_current_price(coin))), 1.00 / float(get_current_price(coin))] # has the format [amount held, price bought at]
# The calculate_ta_indicators function calculates different technical indicators and generates trading signals based on these indicators. The indicators are: EMA, MACD, RSI,
profit_threshold = 0.10 # 10 cents

# cancel open orders
open_orders = r.orders.get_all_open_crypto_orders()
print(f'You have {len(open_orders)} open orders. Cancelling all open orders.')
for order in tqdm(open_orders):
    r.orders.cancel_crypto_order(order['id'])


run_id = 0 # used to determine if the coin has been held for more than 1 run
def calculate_ta_indicators(coins):
    try:
        print(f'[coin] | Δ % | Δ $ | Δ % (24h) | Δ $ (24h)')
        for coin in coins:
            positions_dict = update_portfolio()
            # coin = 'BTC'  # Replace 'BTC' with the coin you are interested in
            amount_held = float(positions_dict.get(coin, 0)) 
            value_of_held = amount_held * float(get_current_price(coin))
            # update basic_orders dictionary
            previous_value = basic_orders.get(coin, [0, 0])[1]
            basic_orders[coin] = [amount_held, value_of_held]

            delta_success = False
            try:
                # print this
                # [coin] | Δ % | Δ $ | Δ % (24h) | Δ $ (24h) 
                
                delta_percentage = round((value_of_held - previous_value) / previous_value * 100, 4)
                delta_value = round(value_of_held - previous_value, 4)
                if verbose_mode and run_id != 0:
                    print(f'{coin} | {delta_percentage}% | {delta_value} | {delta_percentage}% | {delta_value} | {run_id}')
                delta_success = True
                run_id += 1
            except:
                pass

            if delta_success and run_id > 1: 
                
                # with all values rounded to 2 decimal places
                if (value_of_held > previous_value + profit_threshold and value_of_held > 1.50) or \
                    delta_percentage > 2.00 or \
                        delta_percentage < -2.00: # if the percent change has either raised by 2% or dropped by 2% sell the coin
                    # sell coin
                    sell_coin(coin)
            elif delta_success == False: # there was an error so go with basic sells
                if (value_of_held > previous_value + profit_threshold and value_of_held > 1.50):
                    # sell coin
                    sell_coin(coin)
            # Get the historical data for the coin
            historicals = r.get_crypto_historicals(coin, interval='hour', span='week', bounds='24_7', info=None)
            
            # Convert the list of dictionaries to a DataFrame
            df = pd.DataFrame(historicals)
            
            # Convert the 'begins_at' column to datetime and set it as the index
            df['begins_at'] = pd.to_datetime(df['begins_at'])
            df.set_index('begins_at', inplace=True)
            
            # Convert the 'close_price' column to float
            df['close_price'] = df['close_price'].astype(float)
            
            # Calculate the EMA
            # print(f'Calculating EMA for {coin}')
            df['ema'] = df['close_price'].ewm(span=12, adjust=False).mean()
            
            # Calculate the MACD
            # print(f'Calculating MACD for {coin}')
            # macd_obj = ta.MACD(df['close_price'])
            df['macd_line'], df['signal_line'], df['macd_hist'] = ta.macd(df['close_price'])
            
            # Calculate the RSI
            # print(f'Calculating RSI for {coin}')
            df['rsi'] = ta.rsi(df['close_price'])
            
            # Buy Signal
            # If the MACD is greater than the signal line and the RSI is less than 30, then buy
            if df['macd_line'].iloc[-1] > df['signal_line'].iloc[-1] and df['rsi'].iloc[-1] < 30:
                buy_coin(coin, 1.00, df['close_price'].iloc[-1])
                # print(f'Buy Signal for {coin}')
            
            # Sell Signal
            # If the MACD is less than the signal line and the RSI is greater than 70, then sell
            elif df['macd_line'].iloc[-1] < df['signal_line'].iloc[-1] and df['rsi'].iloc[-1] > 70:
                sell_coin(coin)
                print(f'Sell Signal for {coin}')
            
            else:
                pass
    except Exception as e:
        print(f'No Signal for {coin}, due to an error {e}')



# use While True to run the program indefinitely
while True:
    calculate_ta_indicators(coins)
    time.sleep(60*5) # wait for 5 minutes




404 Client Error: Not Found for url: https://api.robinhood.com/marketdata/forex/quotes/None/


TypeError: 'NoneType' object is not subscriptable

# test buy and sell functions
# buy 1.00 USD of BTC
buy_coin(
    coin_name = 'BTC',
    amount = float(1.00),
    current_price = float(r.crypto.get_crypto_quote('BTC')['mark_price'])
    )
print('You bought 1.00 USD of BTC')
# sell 1.00 USD of BTC
sell_coin(
    coin_name = 'BTC'
    )
print('You sold 1.00 USD of BTC')


In [ ]:
!pip install newspaper3k

In [ ]:
!nltk.download('omw-1.4')

In [222]:
import configparser
import feedparser
import matplotlib.pyplot as plt
import nltk
import numpy as np
import pandas as pd
import re
import logging
import requests
import seaborn as sns
import warnings
from bs4 import BeautifulSoup
from datetime import datetime, timezone
from newspaper import Article
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from dateutil import parser
from datetime import datetime

# set up a logging file for the text analysis in logs/ named text_analysis.log and set the logging level to INFO
logging.basicConfig(filename='logs/text_analysis.log', level=logging.INFO)


warnings.filterwarnings("ignore")
nltk.download('punkt')
nltk.download('vader_lexicon')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('omw-1.4')

config = configparser.ConfigParser()
config.read('config/credentials.ini')
coins = config['trading']['coins'].split(',')
apikey =  config['live_coin_watch']['API_KEY']
robinhood_news_urls = {}
reddit_rss_feeds = {}
coinbase_rss_feeds = {}
coin_telegraph_rss_feeds = {}

for coin in coins:
    coin = coin.strip().lower()
    robinhood_news_urls[coin] = config['robinhood_crypto_news_urls'][str(coin)]
    coinbase_rss_feeds[coin] = config['coinbase_rss_feeds'][str(coin)]
    coin_telegraph_rss_feeds[coin] = config['coin_telegraph_rss_feeds'][str(coin)]
    reddit_rss_feeds[coin] = config['reddit_rss_feeds'][str(coin)]

sentiment_analyzer = SentimentIntensityAnalyzer()
sentiment = []
bullish_bearish = []
topic = []

lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))




[nltk_data] Downloading package punkt to
[nltk_data]     /Users/grahamwaters/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/grahamwaters/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/grahamwaters/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/grahamwaters/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/grahamwaters/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/grahamwaters/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [ ]:
def standardize_date(unstructured_date):
    dt = parser.parse(unstructured_date)
    return dt.strftime('%Y-%m-%dT%H:%M:%S%z')

def preprocess_text(text):
    text = re.sub(r'[^a-zA-Z0-9]', ' ', text.lower())
    words = word_tokenize(text)
    words = [word for word in words if word not in stop_words]
    words = [lemmatizer.lemmatize(word) for word in words]
    logging.info(f'Preprocessed text: {len(words)} words; most common: {pd.Series(words).value_counts().head(5).index.tolist()}')
    return ' '.join(words)

def scrape_robinhood_article(url):
    try:
        logging.info(f"Scraping the article at {url}")
        response = requests.get(url)
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')
            article_text = soup.find("div", {"class": "article__content"}).get_text()
            return article_text
        else:
            return None
    except Exception as e:
        logging.error(f"Error scraping the article at {url}: {e}")
        return None

def scrape_robinhood_news_main_feed(url):
    links = []
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    for i in range(10):
        try:
            div = soup.find("div", {"id": f"sdp-news-row-{i}"})
            links.append(div.findChildren("div", recursive=False)[0].findChildren("a", recursive=False)[0]['href'])
        except Exception as e:
            logging.error(f"Error scraping the article at {url}: {e}")
            return None
    logging.info(f"Scraped {len(links)} links from Robinhood Crypto News")
    if links == None:
        links = []
    return links

def sentiment_and_topic_analysis(processed_text):
    sentiment_score = sentiment_analyzer.polarity_scores(processed_text)['compound']
    sentiment.append(sentiment_score)

    bullish_words = ['buy', 'bullish', 'long', 'up']
    bearish_words = ['sell', 'bearish', 'short', 'down']
    num_bullish = len(re.findall(r'\b(?:' + '|'.join(bullish_words) + r')\b', processed_text))
    num_bearish = len(re.findall(r'\b(?:' + '|'.join(bearish_words) + r')\b', processed_text))

    if num_bullish > num_bearish:
        bullish_bearish.append('Bullish')
    elif num_bearish > num_bullish:
        bullish_bearish.append('Bearish')
    else:
        bullish_bearish.append('Neutral')

    vectorizer = CountVectorizer(max_df=0.50, min_df=0.02, max_features=1000, stop_words='english')
    tf = vectorizer.fit_transform([processed_text])
    tf_feature_names = vectorizer.get_feature_names_out()
    num_topics = 1  

    lda_model = LatentDirichletAllocation(n_components=num_topics, max_iter=5, learning_method='online', 
                                            random_state=42, n_jobs=-1)
    lda_model.fit(tf)

    topic_words = [tf_feature_names[i] for i in lda_model.components_[0].argsort()[:-11:-1]]
    topic.append(', '.join(topic_words))

    return sentiment_score, num_bullish, num_bearish, topic_words

def process_feeds(rss_feeds):
    with alive_bar(len(rss_feeds.values())) as bar:
        for rss_url in rss_feeds.values():
            feed = feedparser.parse(rss_url)
            for entry in feed.entries:
                bar.text(f"Processing {entry.title}")
                try:
                    if (datetime.now() - datetime.strptime(standardize_date(entry.published), '%Y-%m-%dT%H:%M:%S+00:00')).total_seconds() < 14400:
                        processed_text = preprocess_text(entry.summary)
                        sentiment_and_topic_analysis(processed_text)
                except Exception as e:
                    logging.error(f"Error processing the article: {e}")
            bar()




df = pd.DataFrame({'Sentiment': sentiment, 'Bullish/Bearish': bullish_bearish,
                   'Suggestion': None, 'Topic': topic}, index=coins)
def generate_suggestion(row):
    if row['Bullish/Bearish'] == 'Bullish' and row['Sentiment'] > 0:
        return 'Buy'
    elif row['Bullish/Bearish'] == 'Bearish' and row['Sentiment'] < 0:
        return 'Sell'
    else:
        return 'Hold'

df['Suggestion'] = df.apply(generate_suggestion, axis=1)

print(df)


Processing BTC articles
Processing ETH articles


ConnectionError: ('Connection aborted.', TimeoutError(60, 'Operation timed out'))

In [ ]:
df.head()

In [ ]:
import nltk
nltk.download('omw-1.4')

In [ ]:
# test buy and sell functions
# buy 1.00 USD of BTC

buy_coin(coin = 'BTC', amount = 1.00, 
# sell 1.00 USD of BTC

In [ ]:
COINTELEGRAPH = {
    'ETH': 'https://cointelegraph.com/rss/tag/ethereum',
    'BTC': 'https://cointelegraph.com/rss/tag/bitcoin',
    'XRP': 'https://cointelegraph.com/rss/tag/xrp',
    'BCH': 'https://cointelegraph.com/rss/tag/bitcoin-cash',
    'LTC': 'https://cointelegraph.com/rss/tag/litecoin',
    'XLM': 'https://cointelegraph.com/rss/tag/stellar',
}

In [223]:
import newspaper
from newspaper import Article
# build a newspaper object
robinhood_paper = newspaper.build('https://cryptonews.com/', memoize_articles=False)
# get the first article
article = robinhood_paper.articles[0]
# download and parse the article
article.download()
article.parse()
# print the article text
print(article.text)

Latest Bitcoin News

Bitcoin is the first peer-to-peer digital currency, also known as cryptocurrency. It is famous for its decentralized transactions, meaning that there is no central governing body operating it, such as a central bank. Bitcoin News will help you to get the latest information about what is happening in the market. Get to know more about “Bitcoin mining” and its circulation tendencies by dedicating some time for the most important Bitcoin News on a daily basis. If you wonder what the future of Bitcoin will look like, stay on the top of the information flow with Bitcoin News today. It’s important to get all the details and to form your own opinion about the possible upcoming scenarios. With Bitcoin News today you will be aware of the latest trends and you will be able to accumulate what is likely to happen. Nevertheless, it’s worth hearing various opinions and create your own overview. Latest Bitcoin News is definitely your needed source of information, so take a look a

In [211]:
# how many articles are there in the dataset?
print(len(robinhood_paper.articles))

1397


In [293]:
import newspaper
from newspaper import Article
# build a newspaper object
crypto_newspaper = newspaper.build('https://cryptonews.com/', memoize_articles=False)
# create a pipeline to (1) download, (2) parse, and (3) NER, and (4) Sentiment Analysis, and (5) identify keywords related to buying or selling in the article texts.
from spacy.pipeline import EntityRuler
from spacy.lang.en import English
from spacy.matcher import Matcher
from spacy.tokens import Span
from spacy import displacy
import spacy
import en_core_web_sm
import pandas as pd

# load the pre-trained model
nlp = en_core_web_sm.load()

# create a pipeline to (1) download, (2) parse, and (3) NER, and (4) Sentiment Analysis, and (5) identify keywords related to buying or selling in the article texts.
def get_article_info(url):
    article = Article(url)
    article.download()
    article.parse()
    article.nlp()
    return article

pipeline = [nlp.create_pipe('sentencizer'),
            nlp.create_pipe('ner'),
            nlp.create_pipe('textcat'),
            nlp.create_pipe('entity_ruler')]

# add the pipeline to the model
for pipe in pipeline:
    nlp.add_pipe(pipe)

# create a list of keywords related to buying or selling
buying_keywords = ['buy', 'bought', 'purchase', 'purchased', 'invest', 'invested', 'investing', 'investor', 'investors', 'investments']

selling_keywords = ['sell', 'sold', 'sale', 'sales', 'selling', 'seller', 'sellers']

# create a function to identify the keywords related to buying or selling in the article texts
def get_buying_selling_keywords(text):
    doc = nlp(text)
    buying = []
    selling = []
    for token in doc:
        if token.text in buying_keywords:
            buying.append(token.text)
        elif token.text in selling_keywords:
            selling.append(token.text)
    return buying, selling

# create a function to get the sentiment score of the article texts
def get_sentiment_score(text):
    doc = nlp(text)
    return doc.cats

# create a function to get the entities in the article texts
def get_entities(text):
    doc = nlp(text)
    return doc.ents

# predict buys and sells
def predict_buys_sells(text):
    buying, selling = get_buying_selling_keywords(text)
    if len(buying) > len(selling):
        return 'buy'
    elif len(buying) < len(selling):
        return 'sell'
    else:
        return 'hold'

# run a loop, pulling the latest articles, and update buy/sell/hold predictions for all coins in coins list
coin_sentiments = {}

while True:
    for article in crypto_newspaper.articles:
        # skip articles that don't include a coin in the text of the article
        if not any(coin in article.text for coin in coins):
            continue
        # skip articles that have already been processed
        if article.url in article_df['url'].values:
            continue
    
        # add the sentiment of the article to the coin mentioned most in the article text
        sentiment = get_sentiment_score(article.text)
        for coin in coins:
            if coin in article.text:
                if coin in coin_sentiments:
                    float(coin_sentiments[coin]) += float([sentiment])

        print(article.title)
    
    # wait for an hour before pulling the latest articles again
    time.sleep(3600)

SyntaxError: 'function call' is an illegal expression for augmented assignment (5210091.py, line 89)